In [ ]:
import pandas as pd
import pymysql
import requests

# Membaca data produk dari file Excel
products_df = pd.read_excel('products.xlsx')

# Membaca data transaksi dari file CSV
transactions_df = pd.read_csv('transactions.csv')

# Membuat koneksi ke database MySQL
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='',
                             database='db_e-commerce')
# Membuat cursor
cursor = connection.cursor()

# Eksekusi SQL untuk mengambil data pelanggan
sql = "SELECT * FROM customers"
cursor.execute(sql)

# Membaca hasil query ke dalam DataFrame
customers_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Menutup koneksi dan cursor
cursor.close()
connection.close()

# Mengambil data pengiriman dari API
response = requests.get('https://fakestoreapi.com/products')
delivery_data = response.json()

# Mengubah data pengiriman ke dalam DataFrame
delivery_df = pd.DataFrame(delivery_data)

# Mengganti nama kolom pada setiap DataFrame sebelum penggabungan
products_df = products_df.rename(columns={'ID': 'Product ID', 'Product Name': 'Product Name', 'Category': 'Product Category', 'Description': 'Product Description', 'Price': 'Product Price'})
transactions_df = transactions_df.rename(columns={'ID': 'Transaction ID', 'User ID': 'User ID', 'Item Name': 'Item Name', 'Quantity': 'Quantity', 'Value (USD)': 'Value (USD)', 'Status': 'Status'})
customers_df = customers_df.rename(columns={'id': 'Customer ID', 'name': 'Customer Name', 'email': 'Customer Email', 'address': 'Customer Address'})
delivery_df = delivery_df.rename(columns={'id': 'Delivery ID', 'title': 'Delivery Title', 'price': 'Delivery Price', 'description': 'Delivery Description', 'category': 'Delivery Category', 'image': 'Delivery Image', 'rating': 'Delivery Rating'})

# Menggabungkan DataFrame dengan nama kolom yang telah diubah
merged_df = pd.concat([products_df, transactions_df, customers_df, delivery_df], axis=1)


# Menyimpan DataFrame ke dalam file Parquet
merged_df.to_parquet('output_file.parquet')

# Baca file Parquet
df = pd.read_parquet('output_file.parquet')

# Tampilkan DataFrame
print(df)